## Imports

In [1]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import spotipy.util as util
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import time 
import statistics 

In [2]:
chart = pd.read_csv('../regional-global-daily-latest.csv', header = 1)
chart.head()

,Position,Track Name,Artist,Streams,URL
0,1,DÁKITI,Bad Bunny,6016657,https://open.spotify.com/track/4MzXwWMhyBbmu6h...
1,2,All I Want for Christmas Is You,Mariah Carey,5902919,https://open.spotify.com/track/0bYg9bo50gSsH3L...
2,3,Last Christmas,Wham!,5153859,https://open.spotify.com/track/2FRnf9qhLbvw8fu...
3,4,Santa Tell Me,Ariana Grande,4202883,https://open.spotify.com/track/0lizgQ7Qw35od7C...
4,5,It's Beginning to Look a Lot like Christmas,Michael Bublé,4074711,https://open.spotify.com/track/0lLdorYw7lVrJyd...


In [3]:
song_uris = []
for url in chart['URL']:
     song_uris.append(url[31:])
        
chart['song_uri'] = song_uris

In [4]:
chart.head(10)

,Position,Track Name,Artist,Streams,URL,song_uri
0,1,DÁKITI,Bad Bunny,6016657,https://open.spotify.com/track/4MzXwWMhyBbmu6h...,4MzXwWMhyBbmu6hOcLVD49
1,2,All I Want for Christmas Is You,Mariah Carey,5902919,https://open.spotify.com/track/0bYg9bo50gSsH3L...,0bYg9bo50gSsH3LtXe2SQn
2,3,Last Christmas,Wham!,5153859,https://open.spotify.com/track/2FRnf9qhLbvw8fu...,2FRnf9qhLbvw8fu4IBXx78
3,4,Santa Tell Me,Ariana Grande,4202883,https://open.spotify.com/track/0lizgQ7Qw35od7C...,0lizgQ7Qw35od7CYaoMBZb
4,5,It's Beginning to Look a Lot like Christmas,Michael Bublé,4074711,https://open.spotify.com/track/0lLdorYw7lVrJyd...,0lLdorYw7lVrJydTINhWdI
5,6,Rockin' Around The Christmas Tree,Brenda Lee,3866794,https://open.spotify.com/track/2EjXfH91m7f8HiJ...,2EjXfH91m7f8HiJN1yQg97
6,7,Jingle Bell Rock,Bobby Helms,3707604,https://open.spotify.com/track/7vQbuQcyTflfCIO...,7vQbuQcyTflfCIOu3Uzzya
7,8,positions,Ariana Grande,3578272,https://open.spotify.com/track/35mvY5S1H3J2QZy...,35mvY5S1H3J2QZyna3TFe0
8,9,Mood (feat. iann dior),24kGoldn,3518616,https://open.spotify.com/track/3tjFYV6RSFtuktY...,3tjFYV6RSFtuktYl3ZtYcq
9,10,It's the Most Wonderful Time of the Year,Andy Williams,3451295,https://open.spotify.com/track/1IcR6RlgvDczfvo...,1IcR6RlgvDczfvoWJSSY2A


## Credentials 

In [5]:
creds= pd.read_csv('../credentials')
SPOTIPY_CLIENT_ID = creds['Client ID'][0]
SPOTIPY_CLIENT_SECRET = creds['Client Secret'][0]
#Shameless redirect to my dance group if you're signing in for the first time
SPOTIPY_REDIRECT_URI= 'https://www.ballethermosoamanecer.com/'
username = ""

In [6]:
client_credentials_manager = SpotifyClientCredentials(client_id= SPOTIPY_CLIENT_ID, 
                                                      client_secret=SPOTIPY_CLIENT_SECRET)

sp = spotipy.Spotify(client_credentials_manager= client_credentials_manager)
scope = 'playlist-read-collaborative user-library-read playlist-read-private playlist-modify-public playlist-modify-private user-read-recently-played user-top-read'
token = util.prompt_for_user_token(username, scope, 
                                   client_id= SPOTIPY_CLIENT_ID, 
                                   client_secret= SPOTIPY_CLIENT_SECRET, 
                                   redirect_uri= SPOTIPY_REDIRECT_URI)
if token:
    sp = spotipy.Spotify(auth=token)
    track_audio_features = []
    artist_uri = []
    artist_name = []
    artist_set = set()
    albums_set = set()
    alb_date = []
    alb_name = [] 
    alb_uri = []
    art_uri = []
    alb_genre = []
    alb_group = []
    loops = 0
    
    
    print(f'There are {len(song_uris)} songs that we found')
    while 20* (loops+1) <= len(song_uris):
        #get audio features in batches of 20
        track_audio_features.extend(sp.audio_features(song_uris[20*loops :20*(loops+1)]))
        
        #getting artist information 
        temp= [i for i in sp.tracks(song_uris[20*loops :20*(loops + 1)])['tracks'] ]
        for index in range(len(temp)):
            to_add = [artist['id'] for artist in temp[index]['artists']]
            artist_uri.append(to_add)
            artist_name.append([artist['name'] for artist in temp[index]['artists']])
            
            #if there was a collab, collect artist information
            if len(to_add) > 1:
                for artist in temp[index]['artists']:
                    artist_set.add(artist['id'])
        loops += 1
        print(f'Found audio features for {20*(loops+1)} songs')
        
    print('Creating Dataframe')
    audio_features = pd.DataFrame(track_audio_features)
    chart['artist_uri'] = artist_uri
    
    print("Merging Dataframe")
    df = chart.merge(audio_features, left_on = 'song_uri', right_on= 'id')
    

else:
    print("Can't get token for", username)
    
    
    

There are 200 songs that we found
Found audio features for 40 songs
Found audio features for 60 songs
Found audio features for 80 songs
Found audio features for 100 songs
Found audio features for 120 songs
Found audio features for 140 songs
Found audio features for 160 songs
Found audio features for 180 songs
Found audio features for 200 songs
Found audio features for 220 songs
Creating Dataframe
Merging Dataframe


In [7]:
type(sp.audio_features(sp.album(sp.artist_albums('4q3ewBCX7sLwd24euuV69X')['items'][0]['id'])['tracks']['items'][0]['id'])[0]
)

dict

In [ ]:
client_credentials_manager = SpotifyClientCredentials(client_id= SPOTIPY_CLIENT_ID, 
                                                      client_secret=SPOTIPY_CLIENT_SECRET)

sp = spotipy.Spotify(client_credentials_manager= client_credentials_manager)
scope = 'playlist-read-collaborative user-library-read playlist-read-private playlist-modify-public playlist-modify-private user-read-recently-played user-top-read'
token = util.prompt_for_user_token(username, scope, 
                                   client_id= SPOTIPY_CLIENT_ID, 
                                   client_secret= SPOTIPY_CLIENT_SECRET, 
                                   redirect_uri= SPOTIPY_REDIRECT_URI)
#     ====================================================================
#                       Generating Artist's Features Data
#     ====================================================================

if token:
    a_count = 0 
    print(f'There are {len(artist_set)} artists that we found')
    aud_feat = []
    
    for uri in artist_set: 
        time.sleep(.3)
        for i in range(len(sp.artist_albums(uri)['items'])):
            art_uri.append(uri)
            alb_date.append(sp.artist_albums(uri)['items'][i]['release_date'])
            alb_name.append(sp.artist_albums(uri)['items'][i]['name']) 
            temp_uri = sp.artist_albums(uri)['items'][i]['id']
            temp_name = sp.artist_albums(uri)['items'][i]['name']
            alb_uri.append(temp_uri)
            albums_set.add(temp_uri)
            alb_group.append(sp.artist_albums(uri)['items'][i]['album_group'])
            
            danceability = []
            energy = []
            key= []
            mode = []
            loudness = []
            speechiness = []
            acousticness = []
            instrumentalness = []   
            liveness = []
            valence = []
            tempo = []
            
            if temp_uri in albums_set:
                print(f'Calculating album features for  {temp_name}' )
                for j in range(len(sp.album((temp_uri))['tracks']['items'])):
                    
                    feature = sp.audio_features(sp.album((temp_uri))['tracks']['items'][j]['id'])[0]
                    if type(feature) == dict: 
                        l1 = []
                        l2 = []
                        l3 = []
                        l4 = []
                        l5 = []
                        l6 = []
                        l7 = []
                        l8 = []
                        l9 = []
                        l10 = []
                        l11 = []

                        l1.append(feature['danceability'])
                        l2.append(feature['energy'])
                        l3.append(feature['key'])
                        l4.append(feature['mode'])
                        l5.append(feature['loudness'])
                        l6.append(feature['speechiness'])
                        l7.append(feature['acousticness'])
                        l8.append(feature['instrumentalness'])
                        l9.append(feature['liveness'])
                        l10.append(feature['valence'])
                        l11.append(feature['tempo'])

                danceability.append(statistics.mean(l1))
                energy.append(statistics.mean(l2))
                key.append(statistics.mode(l3))
                mode.append(statistics.mode(l4))
                loudness.append(statistics.mean(l5))
                speechiness.append(statistics.mean(l6))
                acousticness.append(statistics.mean(l7))
                instrumentalness.append(statistics.mean(l8))
                liveness.append(statistics.mean(l9))
                valence.append(statistics.mean(l10))
                tempo.append(statistics.mean(l11))
            
            else: 
                print(f'Found a duplicate album: {temp_uri}. Will not calculate audio features')
                danceability.append('DUPLICATE')
                energy.append('DUPLICATE')
                key.append('DUPLICATE')
                mode.append('DUPLICATE')
                loudness.append('DUPLICATE')
                speechiness.append('DUPLICATE')
                acousticness.append('DUPLICATE')
                instrumentalness.append('DUPLICATE')   
                liveness.append('DUPLICATE')
                valence.append('DUPLICATE')
                tempo.append('DUPLICATE')
    
        a_count += 1 
        if a_count % 5 == 0:     
            print(f'Found album features for {a_count} artists')
    
    print("Creating Artist Dataframe")
    album_data = pd.DataFrame({'Artist URI': art_uri,
                               'Album URI': alb_uri,
                               'Album Group': alb_group,
                               'Album Name': alb_name,
                               'Album Release Date' : alb_date,
                               'Album Danceability' : danceability,
                               'Album Energy' : energy,
                               'Album Key' : key,
                               'Album Loudness' : loudness,
                               'Album Speechiness' : speechiness,
                               'Album Acousticness' : acousticness,
                               'Album Instrumentalness' : instrumentalness,
                               'Album Liveness' : liveness,
                               'Album Valence': valence,
                               'Album Tempo' : tempo
#                              'Album Generes': alb_genres
                          })
    

There are 129 artists that we found
Calculating album features for  Sunday Drive
Calculating album features for  Glow (Deluxe Edition)
Calculating album features for  Brett Eldredge
Calculating album features for  Glow
Calculating album features for  Illinois
Calculating album features for  Bring You Back
Calculating album features for  Baby, It's Cold Outside (Latin Version)
Calculating album features for  Baby, It's Cold Outside (feat. Sofia Reyes) [Latin Version]
Calculating album features for  Under The Mistletoe
Calculating album features for  When The Party's Over (Live)
Calculating album features for  Crazy Little Thing Called Love
Calculating album features for  Blue Christmas
Calculating album features for  No Stopping You
Calculating album features for  Castaway
Calculating album features for  Love Someone
Calculating album features for  The Long Way
Calculating album features for  Somethin' I'm Good At
Calculating album features for  Baby, It's Cold Outside (feat. Meghan Tra

Calculating album features for  ily (i love you baby) (feat. Emilee)
Calculating album features for  bedroom
Calculating album features for  Taken Away
Calculating album features for  Sleeper (Surf Mesa Remix)
Calculating album features for  Best of 2020 Pop


In [ ]:
# sp.audio_features(sp.album((temp_uri))['tracks']['items'][j]['id'])

In [ ]:
# sp.audio_features('2KdGrc6S5j96mQ7HjKzeHY')

In [ ]:
# album_data[album_data['Album URI'].duplicated()]

In [ ]:
df

In [ ]:
# df['collaborators'] = df['artist_uri'].apply(lambda x : len(x))

In [ ]:
# df[df['collaborators']>1]

In [ ]:
# df.drop(['URL', 'song_uri', 'type', 'uri', 'track_href'])

In [ ]:
df.columns

In [ ]:
album_data